In [0]:
!pip install --upgrade pip
!pip install tensorflow-gpu

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 380.8MB 16kB/s 
     |████████████████████████████████| 3.8MB 25.8MB/s 
     |████████████████████████████████| 450kB 28.3MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstal

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Inception Network
# X = np.load("/content/drive/My Drive/VIDEO_RGB/DataFeatures.npy",allow_pickle=True)
# Y = np.load("/content/drive/My Drive/VIDEO_RGB/Labels.npy",allow_pickle=True)

#ResNetV2
X = np.load("/content/drive/My Drive/VIDEO_RGB/DataFeatures_ResNetV2.npy",allow_pickle=True)
Y = np.load("/content/drive/My Drive/VIDEO_RGB/Labels_ResNetV2.npy",allow_pickle=True)

In [0]:
#taking average over time-steps.
X = np.mean(X, axis=1)
X.shape

(1980, 2048)

In [0]:
X_train = []
Y_train = []

X_val = []
Y_val = []

X_test = []
Y_test = []

for i in range(0,12):
    start = i * 165
    for j in range(0,40):
        X_train.append(X[start + 3*j,:])
        X_train.append(X[start + 3*j+1,:])
        X_train.append(X[start + 3*j+2,:])
        
        Y_train.append(Y[start + 3*j])
        Y_train.append(Y[start + 3*j+1])
        Y_train.append(Y[start + 3*j+2])
        
    for j in range(40,50):
        X_val.append(X[start + 3*j,:])
        X_val.append(X[start + 3*j+1,:])
        X_val.append(X[start + 3*j+2,:])
        
        Y_val.append(Y[start + 3*j])
        Y_val.append(Y[start + 3*j+1])
        Y_val.append(Y[start + 3*j+2])
        
    for j in range(50,55):
        X_test.append(X[start + 3*j,:])
        X_test.append(X[start + 3*j+1,:])
        X_test.append(X[start + 3*j+2,:])
        
        Y_test.append(Y[start + 3*j])
        Y_test.append(Y[start + 3*j+1])
        Y_test.append(Y[start + 3*j+2])
        
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)


In [0]:
X_train.shape

(1440, 2048)

In [0]:
import os
directory = "drive/My Drive/VIDEO_RGB"
entries = os.listdir(directory)

num_labels = 12
LabelToVec = {}
VecToLabel = {}
I = np.identity(num_labels);

for i in range(0,num_labels):
  name = entries[i]
  LabelToVec[name] = I[i,:]
  VecToLabel[i] = name

mask = np.arange(num_labels)

Y_vector = []
Y_tr = []
Y_v = []
Y_te = []

for i in Y_train:
  string = str(i,'utf-8')
  vec = LabelToVec[string]
  Y_tr.append(vec)

for i in Y_val:
  string = str(i,'utf-8')
  vec = LabelToVec[string]
  Y_v.append(vec)

for i in Y_test:
  string = str(i,'utf-8')
  vec = LabelToVec[string]
  Y_te.append(vec)

Y_train = np.asarray(Y_tr)
Y_val = np.asarray(Y_v)
Y_test = np.asarray(Y_te)

In [0]:
Y_val.shape

(360, 12)

In [0]:
import tensorflow as tf
tf.executing_eagerly()

timesteps = 100
dataDim = 2048

from keras import regularizers

Using TensorFlow backend.


In [0]:
model = tf.keras.Sequential()
model.add( tf.keras.layers.Dense(12,input_shape=(dataDim,),kernel_regularizer=regularizers.l2(0.003)) )
model.add( tf.keras.layers.Activation('softmax'))

In [0]:
lr = 0.001 / 0.95

In [0]:
for epocs in range(0,400):

  if (epocs % 50==0) :
    lr = lr * 0.95
    adam = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=True) 
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=[tf.keras.metrics.CategoricalAccuracy()])
     
  print("Iteration Number", epocs)
  model.fit(X_train,Y_train, epochs=1 ,batch_size = 32, shuffle = True, steps_per_epoch = 45)
  out = model(X_val)
  # out = out.numpy()
  out = tf.keras.backend.eval(out)
  l = len(Y_val)
  correct = 0
  total = 0
  for i in range(0,l):
      a = np.argmax(out[i])
      b = np.argmax(Y_val[i])
      if (a==b) : correct = correct + 1
      total = total + 1

  print("\n Validation Accuracy",correct/total)

Iteration Number 0
Train on 1440 samples
1440/1440 [==============================] - 1s 657us/sample - loss: 2.6099 - categorical_accuracy: 0.1021

 Validation Accuracy 0.08888888888888889
Iteration Number 1
Train on 1440 samples
1440/1440 [==============================] - 0s 90us/sample - loss: 2.4487 - categorical_accuracy: 0.1799

 Validation Accuracy 0.19166666666666668
Iteration Number 2
Train on 1440 samples
1440/1440 [==============================] - 0s 89us/sample - loss: 2.3445 - categorical_accuracy: 0.2368

 Validation Accuracy 0.23055555555555557
Iteration Number 3
Train on 1440 samples
1440/1440 [==============================] - 0s 89us/sample - loss: 2.2854 - categorical_accuracy: 0.2347

 Validation Accuracy 0.25833333333333336
Iteration Number 4
Train on 1440 samples
1440/1440 [==============================] - 0s 93us/sample - loss: 2.2043 - categorical_accuracy: 0.2771

 Validation Accuracy 0.21944444444444444
Iteration Number 5
Train on 1440 samples
1440/1440 [==

In [0]:
confusion_matrix = np.zeros((12,12))

out = model(X_test)
# out = out.numpy()
out = tf.keras.backend.eval(out)
l = len(Y_test)
correct = 0
total = 0

for i in range(0,l):
      a = np.argmax(out[i])
      b = np.argmax(Y_test[i])
      confusion_matrix[b][a] += 1

confusion_matrix

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  4.,  0.,  0.,  1.,  8.],
       [ 0.,  1.,  0.,  6.,  1.,  3.,  0.,  0.,  2.,  2.,  0.,  0.],
       [ 0.,  0., 12.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0., 13.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  2.,  0.,  8.,  2.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 15.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  2.,  5.,  5.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  1.,  1.,  3.,  4.],
       [ 0.,  3.,  2.,  0.,  1.,  0.,  0.,  0.,  4.,  4.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1., 13.,  0.,  0.],
       [ 2.,  0.,  0.,  1.,  0.,  1.,  0.,  3.,  0.,  0.,  1.,  7.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  1.,  4.,  8.]])

In [0]:
s = np.sum(confusion_matrix, axis=1)
s

array([15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.])

In [0]:
print(np.sum(confusion_matrix))

180.0


In [0]:
for i in range(12):
  confusion_matrix[i] /= s[i]

confusion_matrix

array([[0.06666667, 0.        , 0.        , 0.06666667, 0.        ,
        0.        , 0.        , 0.26666667, 0.        , 0.        ,
        0.06666667, 0.53333333],
       [0.        , 0.06666667, 0.        , 0.4       , 0.06666667,
        0.2       , 0.        , 0.        , 0.13333333, 0.13333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.8       , 0.        , 0.        ,
        0.06666667, 0.06666667, 0.        , 0.        , 0.06666667,
        0.        , 0.        ],
       [0.        , 0.06666667, 0.        , 0.86666667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.06666667,
        0.        , 0.        ],
       [0.        , 0.06666667, 0.13333333, 0.        , 0.53333333,
        0.13333333, 0.        , 0.        , 0.        , 0.06666667,
        0.06666667, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ,

In [0]:
sum = 0.

for i in range(12) :
  print(VecToLabel[i])
  print(confusion_matrix[i][i])
  sum += confusion_matrix[i][i]

print(sum/12)

kick_service
0.06666666666666667
forehand_slice
0.06666666666666667
backhand2hands
0.8
forehand_volley
0.8666666666666667
forehand_flat
0.5333333333333333
forehand_openstands
1.0
backhand
0.3333333333333333
flat_service
0.3333333333333333
backhand_slice
0.26666666666666666
backhand_volley
0.8666666666666667
smash
0.06666666666666667
slice_service
0.5333333333333333
0.4777777777777777


In [0]:
# confusion_matrix
# np.savetxt("drive/My Drive/RESULTS/CNN/Inception/Confusion_Matrix_Inception_Avg_noLSTM.csv",confusion_matrix,delimiter = ",")
np.savetxt("drive/My Drive/RESULTS/CNN/Resnet/Confusion_Matrix_ResNetV2_Avg_noLSTM.csv",confusion_matrix,delimiter = ",")